### Charting

The chart library is based on [ChartJs](https://www.chartjs.org/). The MeshWeaver classes are wrappers around the javascript data structures defined in ChartJs and provide a fluent API to interface seemingless with C#. 

In [1]:
#r "nuget:MeshWeaver.Connection.Notebook, 2.0.0-preview1"

Installed Packages MeshWeaver.Connection.Notebook, 2.0.0-preview1

Loading extension script from `C:\Users\rolan\.nuget\packages\meshweaver.connection.notebook\2.0.0-preview1\interactive-extensions\dotnet\extension.dib`

Usage:
#!connect mesh --url https://<mesh-weaver-host>/kernel --kernel-name <kernel-name>


In [2]:
#!connect mesh https://localhost:65260/kernel --kernel-name mesh

Kernel added: #!mesh

In [3]:
#r "MeshWeaver.Charting"
using MeshWeaver.Layout;
using MeshWeaver.Mesh;
using static MeshWeaver.Layout.Controls;
using Microsoft.DotNet.Interactive.Formatting;
using static MeshWeaver.Charting.Chart;
using MeshWeaver.Charting;

In [5]:
double ParetoQuantile(double p, double alpha, double x0)
=> x0 * Math.Pow(1 - p, -1 / alpha);

In [34]:
var random = new Random();
var alpha = 1.5;
var x0 = 1.0;
var nSamples = 1000;
var samples = Enumerable.Range(0, nSamples).Select(i => ParetoQuantile(random.NextDouble(), alpha, x0)).OrderBy(x => x).ToArray();

var min = samples.First();
var max = samples.Last();
var nHistogram = 100;
var binWidth = (max - min) / nHistogram;
var histogram = samples.GroupBy(x => Math.Floor((x-min) / binWidth)).Select(g => (g.Key, Count:g.Count())).ToArray();


var barHistogram = DataSet.Bar(histogram.Select(x => x.Count / (double)nSamples), "PDF");
var pdfChart = Chart.Create(barHistogram)  
    .WithLabels(histogram.Select(x => (x.Key * binWidth + min).ToString("F1")))
    .ToControl()
    .WithStyle(style => style.WithWidth("700px").WithHeight("400px"));

pdfChart

In [35]:
var sum = 0.0;
var cdf = histogram.Select(x => (x.Key, Count: sum += x.Count)).ToArray();
var lineCdf = DataSet.Line(cdf.Select(x => x.Count / (double)nSamples), "CDF");

var cdfChart = Chart.Create(lineCdf)
    .WithLabels(cdf.Select(x => (x.Key * binWidth + min).ToString("F1")))
    .ToControl()
    .WithStyle(style => style.WithWidth("700px").WithHeight("400px"));

cdfChart

In [36]:
Chart.Create(barHistogram, lineCdf)
    .WithLabels(cdf.Select(x => (x.Key * binWidth + min).ToString("F1")))
    .ToControl()
    .WithStyle(style => style.WithWidth("700px").WithHeight("400px"))

In [ ]:
Stack
    .WithView(pdfChart)
    .WithView(cdfChart)
    .WithStyle(style => style.WithWidth("700px").WithHeight("800px"))

In [53]:
LayoutGrid
    .WithView(pdfChart)
    .WithView(cdfChart)
